In [7]:
import dotenv

# Get the path and load the .env file
dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)


True

In [8]:
import langchain
import pandas as pd
import numpy as np
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents import create_csv_agent
from langchain_openai import ChatOpenAI
import os
from constants import *

In [9]:
seattle_data = pd.read_csv('data/realistic_seattle_data.csv').dropna().reset_index().drop(columns=['index','Unnamed: 0'])
seattle_data.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price
0,5,3.5,2438.7,sqft,5468.3,sqft,98144,754767.6
1,6,3.0,2162.4,sqft,0.3,acre,98106,882395.8
2,5,4.0,1923.2,sqft,3508.1,sqft,98107,906536.3
3,6,5.0,3729.2,sqft,5080.2,sqft,98199,1850627.0
4,3,4.0,1090.3,sqft,0.9,acre,98107,709016.5


In [10]:
seattle_data = pd.read_csv('data/realistic_seattle_data.csv').dropna().reset_index().drop(columns=['index','Unnamed: 0'])

acre_indices = seattle_data[seattle_data['lot_size_units']=='acre'].index.tolist()
seattle_data.loc[acre_indices,'lot_size']= seattle_data.loc[acre_indices,'lot_size']*43560
seattle_data = seattle_data.drop(['lot_size_units','size_units'],axis=1)

def from_zipcode_to_area(zipcode_of_interest,seattle_zipcode = SEATTLE_ZIPCODES):
    zipcodes = list(seattle_zipcode.values())
    areas = list(seattle_zipcode.keys())
    res = [areas[i] for i in range(len(areas)) if zipcode_of_interest in zipcodes[i]]
    if res==[]:
        res = ['Central/Downtown']
    return res
seattle_data['Area'] = [from_zipcode_to_area(str(zipcode))[0] for zipcode in seattle_data.zip_code.tolist()]
seattle_data['Possibility to negotiate'] = np.random.choice(['High','Middle','Low'],p=[0.1,0.6,0.3],size=len(seattle_data))
seattle_data['Price cut on original price'] = np.array(np.random.choice(np.linspace(0.01,0.05,10),size=len(seattle_data))*seattle_data['price']).astype(int)

In [ ]:
seattle_data.head()

In [11]:
seattle_data.to_csv('data/seattle_estate_processed.csv')

# Define the role-playing instruction for the language model
role_playing_instruction = (
    "You are a knowledgeable real estate agent. Your job is to help the user find "
    "the perfect home, give advice on real estate, negotiate deals, and provide "
    "expert insights on the housing market. The column 'Possibly to negotiate'"
    "tells you how willing you can be to negotiate with the client. Use the Price cut to promote houses with large price cuts."
    "Answer the questions as professionally as possible."
)

# Initialize your agent using the OPENAI_FUNCTIONS agent type
# Include the real estate instruction in the agent's context
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    'seattle_estate_processed.csv',
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    initial_context=role_playing_instruction,  # Add role-playing instruction
)

# Function to run the agent with a real estate query
def ask_real_estate_question(question):
    try:
        # Combine the role-playing instruction with the user's question
        full_prompt = f"{role_playing_instruction}\n\n{question}"
        response = agent.run(full_prompt)
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
question = "I would like a three house bedroom in the south of Seattle. Do you have any good deal?"
answer = ask_real_estate_question(question)
print(answer)


/Users/mpaz/github/woodshed/ai/articles/virtual-property-consultant/venv/lib/python3.9/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:242: UserWarning: Received additional kwargs {'initial_context': "You are a knowledgeable real estate agent. Your job is to help the user find the perfect home, give advice on real estate, negotiate deals, and provide expert insights on the housing market. The column 'Possibly to negotiate'tells you how willing you can be to negotiate with the client. Use the Price cut to promote houses with large price cuts.Answer the questions as professionally as possible."} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_south = df[df['Area'] == 'South']\ndf_three_bedroom = df_south[df_south['beds'] == 3]\ndf_three_bedroom"}`
responded: Sure, I can help you find a good deal on a three-bedroom house in the south of Seattle. Let me filter the dataframe to show you the available options.

      Unnamed: 0  beds  baths    size  lot_size  zip_code      price   Area  \
112          112     3    3.0   581.1    8712.0     98146   431105.6  South   
214          214     3    4.0  1210.3  161172.0     98168   305367.4  South   
252          252     3    4.0  1083.2     554.7     98144   605844.2  South   
339          339     3    3.0  1695.1    4639.6     98118   528945.1  South   
344          344     3    5.5   972.0   34848.0     98178   387043.6  South   
477          477     3    3.0  2034.8    7948.9     98146   340988.3  South   
480          480     3    2.0   815.9    7336.6     98118   447264.9  South   
509      